Copyright **`(c)`** 2022 Giovanni Squillero `<squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  


# Lab 3: ES

## Task

Write agents able to play [*Nim*](https://en.wikipedia.org/wiki/Nim), with an arbitrary number of rows and an upper bound $k$ on the number of objects that can be removed in a turn (a.k.a., *subtraction game*).

The goal of the game is to **avoid** taking the last object.

* Task2.1: An agent using fixed rules based on *nim-sum* (i.e., an *expert system*)
* Task2.2: An agent using evolved rules using ES

## Instructions

* Create the directory `lab2` inside your personal course repository for the course 
* Put a `README.md` and your solution (all the files, code and auxiliary data if needed)

## Notes

* Working in group is not only allowed, but recommended (see: [Ubuntu](https://en.wikipedia.org/wiki/Ubuntu_philosophy) and [Cooperative Learning](https://files.eric.ed.gov/fulltext/EJ1096789.pdf)). Collaborations must be explicitly declared in the `README.md`.
* [Yanking](https://www.emacswiki.org/emacs/KillingAndYanking) from the internet is allowed, but sources must be explicitly declared in the `README.md`.



In [89]:
import logging
from pprint import pprint, pformat
from collections import namedtuple
import random
from copy import deepcopy
from dataclasses import dataclass
from tqdm.notebook import trange
from scipy.special import softmax


## The *Nim* and *Nimply* classes

In [90]:
Nimply = namedtuple("Nimply", "row, num_objects")


In [91]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects


## Sample (and silly) startegies 

In [92]:
def pure_random(state: Nim) -> Nimply:
    """A completely random move"""
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, min(state.rows[row], state._k))
    return Nimply(row, num_objects)


In [93]:
def gabriele(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, min(c + 1, state._k))]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))


In [94]:
def adaptive(state: Nim) -> Nimply:
    """A strategy that can adapt its parameters"""
    genome = {"love_small": 0.5}


In [95]:
import numpy as np


def nim_sum(state: Nim) -> int:
    tmp = np.array([tuple(int(x) for x in f"{c:032b}") for c in state.rows])
    xor = tmp.sum(axis=0) % 2
    return int("".join(str(_) for _ in xor), base=2)


def analize(raw: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = dict()
    for ply in (Nimply(r, o) for r, c in enumerate(raw.rows) for o in range(1, min(c + 1, raw._k))):        
        tmp = deepcopy(raw)
        tmp.nimming(ply)
        cooked["possible_moves"][ply] = nim_sum(tmp)
    return cooked


def optimal(state: Nim) -> Nimply:
    analysis = analize(state)
    # logging.debug(f"analysis:\n{pformat(analysis)}")
    spicy_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns != 0 and ply.num_objects <= state._k]
    if not spicy_moves:
        spicy_moves = [move for move in list(analysis["possible_moves"].keys()) if move.num_objects <= state._k]
    ply = random.choice(spicy_moves)
    return ply


## Oversimplified match

In [96]:
logging.getLogger().setLevel(logging.INFO)

strategy = (pure_random, pure_random)

nim = Nim(7, 2)
logging.info(f"init : {nim}")
player = 0
while nim:
    ply = strategy[player](nim)
    logging.info(f"ply: player {player} plays {ply}")
    nim.nimming(ply)
    logging.info(f"status: {nim}")
    player = 1 - player
logging.info(f"status: Player {player} won!")


INFO:root:init : <1 3 5 7 9 11 13>
INFO:root:ply: player 0 plays Nimply(row=6, num_objects=1)
INFO:root:status: <1 3 5 7 9 11 12>
INFO:root:ply: player 1 plays Nimply(row=4, num_objects=1)
INFO:root:status: <1 3 5 7 8 11 12>
INFO:root:ply: player 0 plays Nimply(row=4, num_objects=2)
INFO:root:status: <1 3 5 7 6 11 12>
INFO:root:ply: player 1 plays Nimply(row=2, num_objects=1)
INFO:root:status: <1 3 4 7 6 11 12>
INFO:root:ply: player 0 plays Nimply(row=3, num_objects=1)
INFO:root:status: <1 3 4 6 6 11 12>
INFO:root:ply: player 1 plays Nimply(row=4, num_objects=1)
INFO:root:status: <1 3 4 6 5 11 12>
INFO:root:ply: player 0 plays Nimply(row=2, num_objects=1)
INFO:root:status: <1 3 3 6 5 11 12>
INFO:root:ply: player 1 plays Nimply(row=5, num_objects=1)
INFO:root:status: <1 3 3 6 5 10 12>
INFO:root:ply: player 0 plays Nimply(row=4, num_objects=1)
INFO:root:status: <1 3 3 6 4 10 12>
INFO:root:ply: player 1 plays Nimply(row=6, num_objects=1)
INFO:root:status: <1 3 3 6 4 10 11>
INFO:root:ply: 

# Task 2.2

We will divide the game into three phases: early game, mid game and late game and we will try to choose the most appropriate move from the strategies we have available. We will use a probability value to decide which strategy to use among the available ones. In our ES the parameters to optimize will be:
- Phase delimiters: the thresholds we use to change which phase of the game we are in.
- Strategy Weights: values between 1 and 10 to compute the probabilities of the strategies to use. We then use the softmax function to normalize these values.



In [97]:
def calculate_moves_ratio(nim: Nim):
  possible_moves = sum([1 for r, c in enumerate(nim.rows) for o in range(1, min(c+1, nim._k))])
  
  initial_nim=Nim(len(nim.rows), nim._k)
  possible_moves_initial = sum([1 for r, c in enumerate(initial_nim.rows) for o in range(1, min(c+1, nim._k))])
  
  return possible_moves / possible_moves_initial



In [98]:
STRATEGIES = [optimal, pure_random, gabriele]
mut_rate: float = (0.1,3)

@dataclass(init=False)
class State:

  n_strategy: int
  phase_thresholds: list[float]
  strategy_weights: list[list[float]]

  def __init__(self, n_strategy: int = None, phase_thresholds: list[float] = None, strategy_weights: list[list[float]] = None) -> None:
    if n_strategy is None:
      n_strategy = len(STRATEGIES)
    if phase_thresholds is None:
      phase_thresholds = sorted([random.random(), random.random()])
    else:
      phase_thresholds = sorted([max(0, phase_thresholds[0]), min(1, phase_thresholds[1])])
    if strategy_weights is None:
      strategy_weights = np.random.uniform(1, 10, (len(phase_thresholds) + 1, n_strategy))
    
    self.n_strategy = n_strategy
    self.phase_thresholds = phase_thresholds
    self.strategy_weights = strategy_weights

  def mutate(state : "State") -> "State":
    global mut_rate
    new_state = deepcopy(state)
    phase_thresholds = np.clip(np.random.normal(state.phase_thresholds, mut_rate[0]), 0, 1).tolist()
    strategy_weights = np.clip(np.random.normal(state.strategy_weights, mut_rate[1]), 1, 10).tolist()
    return State(state.n_strategy, phase_thresholds, strategy_weights)
  
  def __call__(self: "State", state: Nim) -> Nimply:
    phase_ratio = calculate_moves_ratio(state)
    phase = (0 if phase_ratio < self.phase_thresholds[0] else 1 if phase_ratio < self.phase_thresholds[1] else 2)
    probabilities = softmax(self.strategy_weights[phase])
    strategy = np.random.choice(STRATEGIES, p=probabilities)
    return strategy(state)
    
  def __str__(self):
    return f"<Strategies: {self.n_strategy}, Phase Thresholds: {self.phase_thresholds}, Weights: {self.strategy_weights}>"

In [99]:
OPPONENT = optimal
N_GAMES = 25


def match(nim: Nim, strategy: tuple[Nimply]) -> int:
  player = 0
  while nim:
    ply = strategy[player](nim)
    nim.nimming(ply)
    player = 1 - player
  return player

def play_matches(p1_strategy, opp_strategy=OPPONENT, n_games = N_GAMES):
  p1_wins = 0
  for _ in range(n_games):
    random_size = random.randint(4, 10)
    random_k = random.randint(2, 10)
    nim = Nim(random_size, random_k)
    p1_wins +=1 if match(nim, (p1_strategy, opp_strategy)) == 0 else 0
  return p1_wins / n_games

# (1, $\lambda$)-ES

In [101]:
LAMBDA = 10
N_GAMES = 10

solution = State()
solution_score = play_matches(solution)
mut_rate = (0.1, 3)

pbar = trange(0, 1000//LAMBDA)
for _ in pbar:
  pbar.set_description(f"Best score: {solution_score}")
  offspring = [State.mutate(solution) for _ in range(LAMBDA)]
  offspring_scores = [play_matches(offspring) for offspring in offspring]
  # calculate incr rate
  incr_rate = sum([1 for offspring_score in offspring_scores if offspring_score > solution_score]) / LAMBDA
  if incr_rate > 0.2:
    mut_rate = (mut_rate[0] * 1.1, mut_rate[1] * 1.1)
  elif incr_rate < 0.2:
    mut_rate = (mut_rate[0] * 0.9, mut_rate[1] * 0.9)

  # update solution
  solution = offspring[np.argmax(offspring_scores)]
  solution_score = max(offspring_scores)

  if solution_score >= 0.99:
    break 

normalized_weights = softmax(solution.strategy_weights, axis=1)
solution.strategy_weights = normalized_weights.tolist()

print(f"Strategies used: {STRATEGIES}")
print(f"Phase thresholds: {solution.phase_thresholds}")
print(f"Strategy weights in early game: {solution.strategy_weights[0]}")
print(f"Strategy weights in mid game: {solution.strategy_weights[1]}")
print(f"Strategy weights in late game: {solution.strategy_weights[2]}")
print(f"Solution score: {solution_score}")

  0%|          | 0/100 [00:00<?, ?it/s]

Strategies used: [<function optimal at 0x7fdf6157b060>, <function pure_random at 0x7fdf61578d60>, <function gabriele at 0x7fdf61578860>]
Phase thresholds: [0.8094549054991292, 0.9868326943165479]
Strategy weights in early game: [0.9981877542469261, 0.0007272658439016103, 0.0010849799091722115]
Strategy weights in mid game: [0.8629330850186336, 0.011724942145367472, 0.12534197283599893]
Strategy weights in late game: [0.8389051329825785, 0.0005359711490094399, 0.16055889586841204]
Solution score: 0.6
